In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM

In [4]:
# Carga de datos
df = pd.read_csv('..\sell-in.txt', sep='\t')
df['fecha'] = pd.to_datetime(df['periodo'].astype(str), format='%Y%m')

In [5]:
# Función para crear la serie de tiempo para un producto
def serie_producto(id_producto):
    df_prod = df[df['product_id'] == id_producto].loc[:, ['fecha', 'tn']]
    if len(df_prod) == 0:
        return None
    
    # Filtrar solo los períodos de 2019
    df_prod = df_prod[df_prod['fecha'].dt.year == 2018]

    # Encontrar la fecha inicial y final para el producto
    fecha_inicial = df_prod['fecha'].min()
    fecha_final = df_prod['fecha'].max()
    
    # Verificar si fecha_inicial y fecha_final no son NaT
    if pd.isna(fecha_inicial) or pd.isna(fecha_final):
        return None
    
    # Generar fechas hasta febrero de 2020
    fechas_productos = pd.date_range(start=fecha_inicial, end='2020-02-01', freq='MS')
    df_fechas_productos = pd.DataFrame({'fecha': fechas_productos})
    
    df_prod = df_prod.groupby('fecha').agg({'tn': 'sum'}).reset_index()
    df_ret = pd.merge(df_fechas_productos, df_prod, on='fecha', how='left')
    df_ret['tn'].interpolate(method='linear', inplace=True)
    df_ret.fillna(0, inplace=True)
    return df_ret

In [6]:
# Función para crear dataset para LSTM
def create_dataset(data, time_step=1):
    X, Y = [], []
    for i in range(len(data)-time_step-1):
        a = data[i:(i+time_step), 0]
        X.append(a)
        Y.append(data[i + time_step, 0])
    return np.array(X), np.array(Y)

In [7]:
# DataFrame para almacenar predicciones
df_pred = pd.read_csv('..\productos_a_predecir.txt', sep='\t')

# Predicciones
resultados = []

In [10]:
for i in range(len(df_pred)):
    id_producto = df_pred.iloc[i, 0]
    df_prod = serie_producto(id_producto)
    print(f"Progress: {i+1}/{len(df_pred)}", end="\r")
    if df_prod is not None:
        # Preparar datos
        df_prod = df_prod.rename(columns={'fecha': 'ds', 'tn': 'y'})
        dataset = df_prod['y'].values.reshape(-1, 1)
        
        # Normalizar los datos
        scaler = MinMaxScaler(feature_range=(0, 1))
        dataset = scaler.fit_transform(dataset)
        
        # Crear dataset para LSTM
        time_step = 12  # 12 meses (1 año) como ventana de tiempo
        X, Y = create_dataset(dataset, time_step)
        
        # Verificar si X y Y no están vacíos
        if len(X) == 0 or len(Y) == 0:
            print(f"Not enough data for product {id_producto}. Skipping...")
            continue
        
        # Reshape input to be [samples, time steps, features] which is required for LSTM
        X = X.reshape(X.shape[0], X.shape[1], 1)
        
        # Crear y entrenar el modelo LSTM
        model = Sequential()
        model.add(LSTM(5, return_sequences=True, input_shape=(time_step, 1)))
        model.add(LSTM(5, return_sequences=False))
        model.add(Dense(1))
        model.compile(optimizer='adam', loss='mean_squared_error')
        model.fit(X, Y, epochs=25, batch_size=4, verbose=0)
        
        # Predecir el siguiente mes
        last_12_months = dataset[-time_step:]
        last_12_months = last_12_months.reshape(1, time_step, 1)
        pred = model.predict(last_12_months)
        
        # Invertir la normalización
        pred = scaler.inverse_transform(pred)
        pred_final = max(pred[0][0], 0)  # Reemplazar valores negativos por 0
        resultados.append([id_producto, pred_final])

C:\Users\waldo\AppData\Local\Temp\ipykernel_60320\945301539.py:24: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_ret['tn'].interpolate(method='linear', inplace=True)


1/1 [==============================] - 1s 640ms/step


C:\Users\waldo\AppData\Local\Temp\ipykernel_60320\945301539.py:24: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_ret['tn'].interpolate(method='linear', inplace=True)


1/1 [==============================] - 0s 357ms/step


C:\Users\waldo\AppData\Local\Temp\ipykernel_60320\945301539.py:24: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_ret['tn'].interpolate(method='linear', inplace=True)


...

1/1 [==============================] - 0s 357ms/step


C:\Users\waldo\AppData\Local\Temp\ipykernel_60320\945301539.py:24: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_ret['tn'].interpolate(method='linear', inplace=True)


1/1 [==============================] - 1s 772ms/step


C:\Users\waldo\AppData\Local\Temp\ipykernel_60320\945301539.py:24: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_ret['tn'].interpolate(method='linear', inplace=True)


1/1 [==============================] - 0s 358ms/step


In [25]:
df_resultados2 = df_resultados.groupby('product_id').mean().reset_index()

In [26]:
prod_a_predecir = pd.read_csv('..\productos_a_predecir.txt', sep='\t')

In [27]:
df_resultados2 = prod_a_predecir.merge(df_resultados2, on='product_id', how='left')

In [28]:
df_resultados2.fillna(method='ffill', inplace=True)
df_resultados2

C:\Users\waldo\AppData\Local\Temp\ipykernel_60320\42731720.py:1: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_resultados2.fillna(method='ffill', inplace=True)


,product_id,tn
0,20001,1469.863281
1,20002,993.963074
2,20003,764.233459
3,20004,572.759460
4,20005,372.475525
...,...,...
775,21263,0.060342
776,21265,0.060342
777,21266,0.060342
778,21267,0.060342


In [29]:
# Crear DataFrame con los resultados
#df_resultados2 = pd.DataFrame(resultados, columns=['product_id', 'tn'])

# Guardar prediccionesa
df_resultados2.to_csv('pred_lstm.csv',index=False)

: 